In [146]:
import numpy as np
from itertools import chain
import re
from tqdm import tqdm

In [147]:
from math import sqrt

def manhattan(a, b):
    return sum(abs(val1-val2) for val1, val2 in zip(a,b))

In [148]:
with open('aoc15_input_example.txt') as f:
    lines = f.readlines()
lines = [line.strip().split(':') for line in lines]
lines = [[list(map(int,re.findall(r'-?\d+', string))) for string in line] for line in lines]
example_input = lines
example_input
with open('aoc15_input.txt') as f:
    lines = f.readlines()
lines = [line.strip().split(':') for line in lines]
lines = [[list(map(int,re.findall(r'-?\d+', string))) for string in line] for line in lines]
real_input = lines

In [149]:
def flexible_range(i):
    if i == 0:
        return [0]
    elif i > 0:
        return range(i+1)
    else:
        return range(0,i-1,-1)

In [253]:
# lines = example_input
lines = real_input

sensors = [tuple(line[0]) for line in lines]
beacons = [tuple(line[1]) for line in lines]
print(sensors)
print(beacons)

[(3482210, 422224), (3679395, 2737332), (3173475, 3948494), (27235, 3642190), (3851721, 1754784), (327074, 3250656), (3499970, 3186179), (150736, 2522778), (3000768, 3333983), (1751302, 1660540), (2591068, 2923079), (48946, 3999178), (3695475, 3863101), (1504031, 2760), (3021186, 2667125), (1514629, 3771171), (234064, 616106), (3990843, 3393575), (768875, 2665271)]
[(2273934, -202439), (4104213, 2980736), (3494250, 3554521), (-190885, 3635525), (3145586, 2167751), (-190885, 3635525), (3494250, 3554521), (-85806, 2000000), (2564067, 3163630), (3145586, 2167751), (2564067, 3163630), (-190885, 3635525), (3494250, 3554521), (2273934, -202439), (3145586, 2167751), (2564067, 3163630), (-85806, 2000000), (4104213, 2980736), (-85806, 2000000)]


In [179]:
def get_coords(puzzle_input):
    all_coords = puzzle_input
    min_x = min([coord[0] for coord in all_coords])
    min_y = min([coord[1] for coord in all_coords])
    max_x = max([coord[0] for coord in all_coords])
    max_y = max([coord[1] for coord in all_coords])
    return min_x, max_x, min_y, max_y
get_coords(sensors)

(0, 20, 0, 20)

In [180]:
get_coords(beacons)

(-2, 25, 3, 22)

In [153]:
soln_row = 2000000
sensor_set = {tuple(coord) for coord in sensors}
beacon_set = {tuple(coord) for coord in beacons}
excluded_ranges = set([])
excluded_beacons = set([])
for coord in sensor_set.union(beacon_set):
    if coord[1] == soln_row:
        excluded_beacons.add(coord)
num_excluded_beacons = len(excluded_beacons)
# print(num_excluded_beacons)
for beacon, sensor in zip(beacons,sensors):
    mdist = abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1])
    vdist_from_soln = abs(sensor[1] - soln_row)
#     print(sensor, "Y distance from",beacon,'is',mdist)
#     print("Row", soln_row, "distances from",sensor,'is',vdist_from_soln)
    if vdist_from_soln <= mdist:
        xrange_soln = mdist - vdist_from_soln
#         print("Additional manhattan distance for",sensor,'is',xrange_soln)
        excluded_ranges.add((int(sensor[0]-xrange_soln),int(sensor[0]+xrange_soln)))
#         print("Row", soln_row,  " X Range is:", (int(sensor[0]-xrange_soln),int(sensor[0]+xrange_soln)))
# print(excluded_ranges)
excluded_columns = set()
for xrange in excluded_ranges:
#     print(xrange)
    for i in range(xrange[0],xrange[1]+1):
        excluded_columns.add(i)
# print(excluded_columns)
print(len(excluded_columns) - num_excluded_beacons)

4811413
4811413
4811413
4811413
4811413
4811413
4811413
4811413
831 ms ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Part 2

In [254]:

def is_beacon(coord):
    if coord in sensor_set or coord in beacon_set:
        return False
    for sensor,distance in zip(sensors,distances):
        if distance >= abs(sensor[0] - coord[0]) + abs(sensor[1] - coord[1]):
#             print(sensor)
#             print(beacon)
#             print(distance)
#             print(abs(sensor[0] - coord[0]) + abs(sensor[1] - coord[1]))
            return False
    return True

In [255]:
from tqdm import tqdm
distances = []
for beacon, sensor in zip(beacons,sensors):
    distances.append(abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1]))
# print(beacons,sensors,distances)
non_overlapping_sensors = {}
for sensor1, distance1 in zip(sensors,distances):
    non_overlapping_sensors[sensor1] = []
    for sensor2, distance2 in zip(sensors,distances):
        if sensor1 != sensor2:
            mdist = sum(abs(np.array(sensor1)-np.array(sensor2)))
#             print(mdist - (distance1 + distance2))
            if mdist - distance1 - distance2 < 0:
                non_overlapping_sensors[sensor1].append(sensor2)
                
non_overlapping_sensors

{(3482210, 422224): [(3851721, 1754784),
  (1751302, 1660540),
  (1504031, 2760),
  (234064, 616106)],
 (3679395, 2737332): [(3851721, 1754784),
  (3499970, 3186179),
  (3695475, 3863101),
  (3021186, 2667125),
  (3990843, 3393575)],
 (3173475, 3948494): [(3000768, 3333983),
  (3695475, 3863101),
  (1514629, 3771171)],
 (27235, 3642190): [(327074, 3250656),
  (48946, 3999178),
  (1514629, 3771171),
  (768875, 2665271)],
 (3851721, 1754784): [(3482210, 422224),
  (3679395, 2737332),
  (1751302, 1660540)],
 (327074, 3250656): [(27235, 3642190),
  (150736, 2522778),
  (48946, 3999178),
  (1514629, 3771171),
  (768875, 2665271)],
 (3499970, 3186179): [(3679395, 2737332),
  (3000768, 3333983),
  (3695475, 3863101),
  (3990843, 3393575)],
 (150736, 2522778): [(327074, 3250656),
  (1751302, 1660540),
  (234064, 616106),
  (768875, 2665271)],
 (3000768, 3333983): [(3173475, 3948494),
  (3499970, 3186179),
  (2591068, 2923079),
  (3021186, 2667125),
  (1514629, 3771171),
  (3990843, 3393575)],


In [256]:
iterations = []
window = 4000000
for sensor, distance in tqdm(zip(sensors,distances)):
#     print(sensor,distance)
    perimeter = distance + 1
    for p in range(0,perimeter+1):
        i = sensor[0] - perimeter + p
        j = sensor[1] - p
        if 0 < i < window and 0 < j < window:
            if is_beacon((i,j)):
                print((i,j))
                break
        i = sensor[0] + perimeter - p
        j = sensor[1] - p
        if 0 < i < window and 0 < j < window:
            if is_beacon((i,j)):
                print((i,j))
                break
        
        i = sensor[0] - perimeter + p
        j = sensor[1] + p
        if 0 < i < window and 0 < j < window:
            if is_beacon((i,j)):
                print((i,j))
                break
        i = sensor[0] + perimeter - p
        j = sensor[1] + p
        if 0 < i < window and 0 < j < window:
            if is_beacon((i,j)):
                print((i,j))
                break
        

2it [00:12,  5.65s/it]

(3292963, 3019123)


7it [00:37,  4.84s/it]

(3292963, 3019123)


9it [00:48,  5.14s/it]

(3292963, 3019123)


9it [01:07,  7.53s/it]


KeyboardInterrupt: 

In [257]:
3292963*4000000 + 3019123

13171855019123

In [245]:
grid = np.chararray((50,50),unicode=True)
grid[:] = '.'

In [ ]:
3292963*4000000 + 3019123

In [246]:
for beacon in beacons:
    i,j = np.array(beacon)
    grid[j][i] = 'B'
for sensor in sensors:
    i,j = np.array(sensor)
    grid[j][i] = 'S'
np.savetxt('aoc15_output_example.txt',grid,fmt='%s')


In [248]:
for beacon, sensor,in zip(beacons,sensors):
#     print(sensor,beacon)
    mdist = abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1])
#     print(mdist)
    for j in range(sensor[1] - mdist -5,sensor[1]+mdist+5):
        for i in range(sensor[0] - mdist -5,sensor[0]+mdist+5):
            location = np.array([i,j])
            if (sum(abs(sensor - location)) <= mdist) and grid[j][i] == '.':
                grid[j][i] = '#'

iterations = []
window = 20
for sensor, distance in tqdm(zip(sensors,distances)):
    print(sensor,distance)
    perimeter = distance + 1
    for p in range(0,perimeter+1):
        i = sensor[0] - perimeter + p
        j = sensor[1] - p
        grid[j][i] = 'X'
        i = sensor[0] + perimeter - p
        j = sensor[1] - p
        grid[j][i] = 'X'
        
        i = sensor[0] - perimeter + p
        j = sensor[1] + p
        grid[j][i] = 'X'
        i = sensor[0] + perimeter - p
        j = sensor[1] + p
        grid[j][i] = 'X'

        
np.savetxt('aoc15_output_example.txt',grid,fmt='%s')

14it [00:00, 18862.92it/s]

(2, 18) 7
(9, 16) 1
(13, 2) 3
(12, 14) 4
(10, 20) 4
(14, 17) 5
(8, 7) 9
(2, 0) 10
(0, 11) 3
(20, 14) 8
(17, 20) 6
(16, 7) 5
(14, 3) 1
(20, 1) 7


In [ ]:
sensor, beacon, distance =np.array([8, 7]), np.array([ 2, 10]), np.array([ 6, -3])
print(sensor,beacon,np.array(distance))
mdist = sum(abs(distance))
print(mdist)
for j in range(len(grid)):
    for i in range(len(grid[j])):
        location = np.array([i,j])
        if (sum(abs(sensor - location)) <= mdist) and grid[j][i] == '.':
            grid[j][i] = '#'

In [ ]:
np.savetxt('aoc15_output_example.txt',grid,fmt='%s')

In [28]:
for row in grid:
    print(sum(np.char.count(row, '#')))

34
34
32
29
29
27
25
21
23
25
26
27
28
26
22
21
20
20
23
22
19
17
13
10
6
3
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
3
5
7
10
14
18
22
26
31


In [66]:
def search_space(max):
    for i in range(max):
        for j in range(max):
            yield (i,j)


In [86]:
i = search_space(3)
next(i)

(0, 0)

StopIteration: 